# Introduction to Discrete-Event Modeling

In [1]:
!pip install simpy

## A simple simulation model

As a simple demonstration, let's consider the task of modeling the progess of a disease in a single person. The person will go through several stages of the disease

$$\text{Uninfected} \quad \longrightarrow \quad \text{Exposed} \quad \longrightarrow \quad \text{Infectious} \quad \longrightarrow \quad \text{Recovered}$$

### Python generators

Python generators are the basic building block for discrete event simulation in Python. Python generators are similar to functions with three profound differences:

1. They return values using the `yield` statement rather than a `return` statement. They don't disappear from memory until all statements have been executed.
2. Because they don't disappear, they can start and be restarted with a `next` statement.
3. Multiple instances of a generator may be working at the same time.

In [24]:
def a_function():
    return 3

a_function()

3

In [25]:
def a_generator():
    yield 3
    
a_generator()

<generator object a_generator at 0x7fd645f02450>

Using a generator is different from a function because they don't disappear from memory

In [27]:
a = a_generator()
next(a)

3

Let's create a generator that can return multiple values.

In [34]:
def my_first_generator():
    yield 1
    yield 2
    yield 3
    yield 4

a = my_first_generator()
print(next(a))

1


Let's get the next value.

In [35]:
print(next(a))

2


Let's create a second instance of the same generator.

In [36]:
b = my_first_generator()
print(next(b))

1


Note how the instances each retain a unique state.

In [38]:
print(next(a))
print(next(b))

4
2


#### Exercise

Create a generator that produces a sequence of successive squares.

In [41]:
def succ_sq():
    k = 0
    while True:
        yield k**2
        k = k + 1
        
a = succ_sq()
print(next(a))
print(next(a))
print(next(a))
print(next(a))

0
1
4
9


#### Exercise

Create generator that creates the Fibonacci sequence

$$\begin{align*}
F_0 & = 0 \\
F_1 & = 1 \\
F_{n} &= F_{n-1} + F_{n-2}
\end{align*}$$

In [45]:
def fib():
    a = 0
    b = 1
    yield a
    yield b
    while True:
        a, b = b, a+b
        yield b
        
f = fib()
for k in range(0, 20):
    print(next(f))

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181


#### Exercise

Write a Python generator that simulates the response of a differential equation describing concentration in a stirred tank reactor.

$$\frac{dC}{dt} = -k C + q(t)$$

where k = 1.0,  C(0) = 1.0 and q(t) = 0.5. Use Euler's approximation for this example.

In [73]:
q = 0.5

def reactor(dt, k):
    C = 1.0
    t = 0.0
    while t <= 5.0:
        yield t, C
        t = t + dt
        C = C - k*C*dt + q*dt
        
a = reactor(0.2, 1.0)
for t, C in a:
    print(round(t, 2), round(C, 3))

0.0 1.0
0.2 0.9
0.4 0.82
0.6 0.756
0.8 0.705
1.0 0.664
1.2 0.631
1.4 0.605
1.6 0.584
1.8 0.567
2.0 0.554
2.2 0.543
2.4 0.534
2.6 0.527
2.8 0.522
3.0 0.518
3.2 0.514
3.4 0.511
3.6 0.509
3.8 0.507
4.0 0.506
4.2 0.505
4.4 0.504
4.6 0.503
4.8 0.502


## Simpy

### Simplest possible Simpy model

Simpy implements a simulation Environment() object that schedules all of the events and processing that occurs when building complex models.

In [3]:
import simpy

env = simpy.Environment()
env.run(until=20)

### Simpy models are about events

In [61]:
import simpy

def clock(dt):
    while True:
        print(env.now)
        yield env.timeout(dt)    

env = simpy.Environment()
a = clock(1.0)
env.process(a)
env.run(until=20)

0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0


#### Exercise

Implement the reactor model using simpy.

In [65]:
import simpy

q = 0.5

def reactor(dt, k):
    C = 1.0
    while True:
        print(round(env.now, 2), round(C, 2))
        yield env.timeout(dt)
        C = C - k*dt*C + q*dt
        
env = simpy.Environment()
env.process(reactor(0.1, 1.0))
env.run(until=5)

0 1.0
0.1 0.95
0.2 0.91
0.3 0.86
0.4 0.83
0.5 0.8
0.6 0.77
0.7 0.74
0.8 0.72
0.9 0.69
1.0 0.67
1.1 0.66
1.2 0.64
1.3 0.63
1.4 0.61
1.5 0.6
1.6 0.59
1.7 0.58
1.8 0.58
1.9 0.57
2.0 0.56
2.1 0.55
2.2 0.55
2.3 0.54
2.4 0.54
2.5 0.54
2.6 0.53
2.7 0.53
2.8 0.53
2.9 0.52
3.0 0.52
3.1 0.52
3.2 0.52
3.3 0.52
3.4 0.51
3.5 0.51
3.6 0.51
3.7 0.51
3.8 0.51
3.9 0.51
4.0 0.51
4.1 0.51
4.2 0.51
4.3 0.51
4.4 0.5
4.5 0.5
4.6 0.5
4.7 0.5
4.8 0.5
4.9 0.5
5.0 0.5


### Modeling with Python Classes

The new problem that we have is accessing the value of C outside the context of the generator. We'd like to do this because other process units may depend on that concentration.

In [82]:
import simpy

q = 0.5

class Reactor():
    def __init__(self, k, dt):
        self.k = k
        self.dt = dt
        self.C = 1.0
        
    def run(self):
        while True:
            print(round(env.now, 2), round(self.C, 2))
            yield env.timeout(self.dt)
            self.C = self.C - self.k*self.C*self.dt + q*self.dt

env = simpy.Environment()
a = Reactor(1.0, 0.2)
b = Reactor(1.4, 0.1)
env.process(a.run())
env.process(b.run())
env.run(until=10)

0 1.0
0 1.0
0.1 0.91
0.2 0.9
0.2 0.83
0.3 0.77
0.4 0.82
0.4 0.71
0.5 0.66
0.6 0.62
0.6 0.76
0.7 0.58
0.8 0.55
0.8 0.7
0.9 0.52
1.0 0.5
1.0 0.66
1.1 0.48
1.2 0.63
1.2 0.46
1.3 0.45
1.4 0.6
1.4 0.43
1.5 0.42
1.6 0.58
1.6 0.41
1.7 0.41
1.8 0.57
1.8 0.4
1.9 0.39
2.0 0.55
2.0 0.39
2.1 0.38
2.2 0.54
2.2 0.38
2.3 0.38
2.4 0.53
2.4 0.37
2.5 0.37
2.6 0.53
2.6 0.37
2.7 0.37
2.8 0.52
2.8 0.37
2.9 0.37
3.0 0.52
3.0 0.36
3.1 0.36
3.2 0.51
3.2 0.36
3.3 0.36
3.4 0.51
3.4 0.36
3.5 0.36
3.6 0.51
3.6 0.36
3.7 0.36
3.8 0.51
3.8 0.36
3.9 0.36
4.0 0.51
4.0 0.36
4.1 0.36
4.2 0.5
4.2 0.36
4.3 0.36
4.4 0.36
4.4 0.5
4.5 0.36
4.6 0.36
4.6 0.5
4.7 0.36
4.8 0.36
4.8 0.5
4.9 0.36
5.0 0.36
5.0 0.5
5.1 0.36
5.2 0.36
5.2 0.5
5.3 0.36
5.4 0.36
5.4 0.5
5.5 0.36
5.6 0.36
5.6 0.5
5.7 0.36
5.8 0.36
5.8 0.5
5.9 0.36
6.0 0.36
6.0 0.5
6.1 0.36
6.2 0.36
6.2 0.5
6.3 0.36
6.4 0.36
6.4 0.5
6.5 0.36
6.6 0.36
6.6 0.5
6.7 0.36
6.8 0.36
6.8 0.5
6.9 0.36
7.0 0.36
7.0 0.5
7.1 0.36
7.2 0.36
7.2 0.5
7.3 0.36
7.4 0.36
7.4 0.5
7.5 0.36
7.

In [79]:
import simpy

q = 0.5

def historian(dt):
    while True:
        print(env.now, a.C)
        yield env.timeout(dt)

class Reactor():
    def __init__(self, k, dt):
        self.k = k
        self.dt = dt
        self.C = 1.0
        
    def run(self):
        while True:
            yield env.timeout(self.dt)
            self.C = self.C - self.k*self.C*self.dt + q*self.dt

env = simpy.Environment()
a = Reactor(1.0, 0.2)
env.process(a.run())
env.process(historian(0.3))
env.run(until=10)

0 1.0
0.3 0.9
0.6 0.82
0.8999999999999999 0.7047999999999999
1.2 0.6638399999999999
1.5 0.6048575999999999
1.8 0.5671088639999999
2.1 0.5536870911999999
2.4 0.54294967296
2.6999999999999997 0.5274877906944
2.9999999999999996 0.52199023255552
3.2999999999999994 0.5140737488355328
3.599999999999999 0.5112589990684262
3.899999999999999 0.5072057594037928
4.199999999999999 0.5057646075230342
4.499999999999999 0.5036893488147418
4.799999999999999 0.5029514790517935
5.099999999999999 0.5018889465931478
5.399999999999999 0.5015111572745182
5.699999999999998 0.5009671406556917
5.999999999999998 0.5007737125245534
6.299999999999998 0.5004951760157141
6.599999999999998 0.5003961408125713
6.899999999999998 0.5002535301200457
7.1999999999999975 0.5002028240960366
7.499999999999997 0.5001298074214634
7.799999999999997 0.5001038459371707
8.099999999999998 0.5000664613997893
8.399999999999999 0.5000531691198314
8.7 0.5000340282366922
9.0 0.5000272225893537
9.3 0.5000174224571864
9.600000000000001 0.5

### COVID Model

In [ ]:
import simpy

import random

class person()
    print
    yield env.timeout(random.uniform(0, 120))